In this Notebook we create and sketch tools for the Agent. Building proofs of concept (PoC)

### Setup

In [1]:
# Libraries

import os

In [2]:
# import Whisper

import whisper

In [3]:
# Import Agent

os.sys.path.append("../../src")
os.sys.path.append("../../src/agents")

import react  # My AI assistant

Langfuse client is disabled since no public_key was provided as a parameter or environment variable 'LANGFUSE_PUBLIC_KEY'. See our docs: https://langfuse.com/docs/sdk/python/low-level-sdk#initialize-client


In [4]:
# DEBUG

os.sys.path.append("../../src/tools")

In [5]:

# Import GAIA Questions
from datasets import load_dataset


/home/santiagoal/.cache/pypoetry/virtualenvs/chappie-CHLGiFC_-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:

import pandas as pd


In [7]:
from dotenv import load_dotenv
from huggingface_hub import snapshot_download, login

In [8]:
# Load Hugging face credentials

#load_dotenv()
#login(os.getenv(key="HF_TOKEN_CHAPPIE"))  # Replace with your hf api key name


In [9]:
#gaia_questions_path = snapshot_download(repo_id="gaia-benchmark/GAIA", repo_type="dataset")
#gaia_questions = load_dataset(path="gaia-benchmark/GAIA", name="2023_level1")

In [10]:
gaia_index_dir = "../../../../.cache/huggingface/datasets/gaia-benchmark___gaia/2023_level1/0.0.1/ec492fe4320ee795b1aed6bb46229c5f693226b0f1316347501c24b4baeee005"
#gaia_index_dir = os.path.abspath(gaia_index_dir)
gaia_data_path = os.path.join(gaia_index_dir, "gaia-validation.arrow")

In [11]:
# Temporal cell: wrote because of hugging face api calls limit

import pyarrow.ipc as ipc

with open(gaia_data_path, "rb") as f:
    reader = ipc.RecordBatchStreamReader(f)
    table = reader.read_all()

gaia_df = table.to_pandas()


In [12]:
#gaia_questions = gaia_questions["validation"]  # Filter for dev purposes
#gaia_df = pd.DataFrame(gaia_questions)

In [13]:
gaia_df.head()

,task_id,Question,Level,Final answer,file_name,file_path,Annotator Metadata
0,e1fc63a2-da7a-432f-be78-7c4a95598703,If Eliud Kipchoge could maintain his record-ma...,1,17,,,{'Steps': '1. Googled Eliud Kipchoge marathon ...
1,8e867cd7-cff9-4e6c-867a-ff5ddc2550be,How many studio albums were published by Merce...,1,3,,,{'Steps': '1. I did a search for Mercedes Sosa...
2,ec09fa32-d03f-4bf8-84b0-1f16922c3ae4,Here's a fun riddle that I think you'll enjoy....,1,3,,,{'Steps': 'Step 1: Evaluate the problem statem...
3,5d0080cb-90d7-4712-bc33-848150e917d3,What was the volume in m^3 of the fish bag tha...,1,0.1777,,,"{'Steps': '1. Searched '""Can Hiccup Supply Eno..."
4,a1e91b78-d3d8-4675-bb8d-62741b4b68a6,In the video https://www.youtube.com/watch?v=L...,1,3,,,{'Steps': '1. Navigate to the YouTube link. 2....


In [14]:
# DEL
# temp filter to get chess images
gaia_df[gaia_df["file_path"].map(lambda f: f.endswith("44.png"))]

,task_id,Question,Level,Final answer,file_name,file_path,Annotator Metadata
16,cca530fc-4052-43b2-b130-b30968d8aa44,Review the chess position provided in the imag...,1,Rd5,cca530fc-4052-43b2-b130-b30968d8aa44.png,/home/santiagoal/.cache/huggingface/hub/datase...,{'Steps': 'Step 1: Evaluate the position of th...


---

In [15]:
filetypes = {d[1].file_path.split(".")[-1] for d in gaia_df.iterrows()}
filetypes

{'', 'docx', 'mp3', 'png', 'pptx', 'py', 'txt', 'xlsx'}

### Read Historical XPs

In [ ]:
xp_paths = "../data/agent_experiments/iterations/"

In [ ]:
# Index all XPs
import os
import pandas as pd

xp_dir = "../../data/agent_experiments/iterations"
xp_list = []
# FIXME: from the 10th XP iteration, the sorted method is unuseful

for i, xp_path in enumerate(sorted(os.listdir(xp_dir))):  
    xp_path = os.path.join(xp_dir, xp_path)
    temp_xp_df = pd.read_csv(xp_path)
    xp_list.append((i, temp_xp_df))
    del temp_xp_df 

In [ ]:
# Join latest XP result to each question

index = gaia_df.index.tolist()  # Index for all the questions
questions = gaia_df["Question"].tolist()
answers = list()


def filter_condition(xp_data: list) -> bool:
    i, xp = xp_data
    for question in questions:
        xp_addressed_questions = xp["Question"].tolist()
        if question in xp_addressed_questions:
            return True
    return False

filtered_xps = list(filter(filter_condition, xp_list))
filtered_xps_reversed = filtered_xps
filtered_xps_reversed.reverse()

# Join latest answer result (i.e. result of the latest xp)

# Join latest answer result (i.e. result of the latest xp)

for question in questions:
    answer_result = 0  # Assume wrong answer by default
    for i, xp in filtered_xps_reversed:
        for xp_question in xp["Question"].tolist():
            
            if question == xp_question:
                answer_result_row = list(xp[xp["Question"] == xp_question]["is_correct"])#.loc[0]
                answer_result_temp = answer_result_row[0]                    
                if answer_result_temp >= answer_result:
                    answer_result = answer_result_temp
        del xp
    answers.append(answer_result)

# Join answers
historical_xp_results = gaia_df.copy()
historical_xp_results["is_correct"] = pd.Series(answers)
del answers, questions

In [ ]:
historical_xp_results

In [ ]:
historical_xp_results.is_correct.mean()

Let's study first which are the most common wrong tasks. I.e. How can we increase accuracy with a single next step (e.g. implement a new tool, modify sys message, etc.)

In [ ]:
wrong_ans_df = historical_xp_results[historical_xp_results["is_correct"]==0]
wrong_ans_df["fp_extension"] = wrong_ans_df["file_path"].map(lambda path: path.split(".")[-1])
wrong_ans_df

In [ ]:
# Summary
wrong_ans_df.groupby("fp_extension")["is_correct"].count()

The vast majority of remaining tasks do not include files to read. So we sould study them at first

In [ ]:
wrong_ans_df_no_extension = wrong_ans_df[wrong_ans_df["file_path"].map(lambda fp: len(fp)==0)]

In [ ]:
wrong_ans_df_no_extension

### Method

We identified three main necessities to cover through web search

1. Analize youtube videos (transcription and computer vision)
2. Scrap web pages (e.g. journals, pdfs, wikipedia articles)


So we decided to modularize the web_search_tool into three main modules

1. High-level Web results
2. Youtube Video Class: To gather both audio and video
3. Web page scrapper: Given a web url, retrieve the text as markdown
4. Object detection tool: Ideally coded into handle_image tool

So we plan to generate three main tools: `youtube_tool` (Class + Langchain runnable), `web_search` (Expanded) and `handle_images` with an initial module `object_detection` using YoloV4-tiny

### Youtube Tool

In [16]:
import re

#### 1. FIlter tasks to solve with the tool

In [17]:
pattern = r".*youtube.*"
yt_tasks = gaia_df.copy()
yt_tasks = yt_tasks[yt_tasks["Question"].map(lambda question: bool(re.search(pattern, question, re.IGNORECASE)))]
yt_tasks

,task_id,Question,Level,Final answer,file_name,file_path,Annotator Metadata
4,a1e91b78-d3d8-4675-bb8d-62741b4b68a6,In the video https://www.youtube.com/watch?v=L...,1,3,,,{'Steps': '1. Navigate to the YouTube link. 2....
26,9d191bce-651d-4746-be2d-7ef8ecadb9c2,Examine the video at https://www.youtube.com/w...,1,Extremely,,,{'Steps': '1. Follow the link 2. Watch the cli...
33,0383a3ee-47a7-41a4-b493-519bdefe0488,On the BBC Earth YouTube video of the Top 5 Si...,1,Rockhopper penguin,,,"{'Steps': '1. Search ""top 5 silliest animal mo..."


#### 2. Tool PoC

In [18]:
sample_yt_url = "https://www.youtube.com/watch?v=L1vXCYZAYYM"

Lets import that video

In [19]:
import os

In [20]:
output_dir = "../../data/temp/"

yt_audio_filename = "yt_audio.mp3"
yt_audio_path = os.path.join(output_dir, yt_audio_filename)

raw_yt_video_filename = "raw_yt_video.mp4"
processed_yt_video_filename = "processed_yt_video.mp4"

raw_yt_video_path = os.path.join(output_dir, raw_yt_video_filename)
processed_yt_video_path = os.path.join(output_dir, processed_yt_video_filename)

In [21]:
from pytubefix import YouTube

In [22]:
yt = YouTube(sample_yt_url)
#yt_audio = yt.streams.filter(only_audio=True).first()
#yt_video = yt.streams.filter(only_video=True).first()

In [23]:
yt_audio = yt.streams.filter(only_audio=True).first()
yt_audio_filename = "yt_audio.mp3"
yt_audio_path = os.path.join(output_dir, yt_audio_filename)

In [24]:
yt_video = yt.streams.filter(only_video=True, fps=25, res="144p").order_by("fps").asc().first()
yt_video_filename = "yt_video.mp4"
yt_video_path = os.path.join(output_dir, raw_yt_video_filename)

We imported a binary representation of both YT video and audio. The idea now is to try to pass these objects to a CV model (e.g. YoloV4) and to a Transcriber model (e.g. Whisper)

YoLo

In [ ]:
yt_video.download(output_path=output_dir, filename=raw_yt_video_filename)

In [ ]:
import subprocess

new_fps = 1
reduce_fps_cmd = f'ffmpeg -y -i {raw_yt_video_path} -filter:v "fps={new_fps}" -an {processed_yt_video_path}'
subprocess.run(
    reduce_fps_cmd, shell=True
)

In [ ]:
from ultralytics import YOLO

cv_model = YOLO(model="yolov8s.pt", task="detect")

In [ ]:
cv_results = cv_model.predict(source=processed_yt_video_path, stream=True)

In [ ]:
os.sys.getsizeof(cv_results)

In [ ]:
def summarize(results):# -> Generator[dict[str, Any], Any, None]:# -> Generator[dict[str, Any], Any, None]:
    for i, result in enumerate(results):
        yield {
            "frame": i + 1,
            "result": [result.names[int(cls)] for cls in result.boxes.cls]
           }

In [ ]:
data = list(summarize(cv_results))

In [ ]:
# TODO: make this a typed dict to let the agent process this data
processed_data = {
    "video_url": sample_yt_url,
    "detected_objects": data
}

In [ ]:
processed_data

In [ ]:
# delete unnecesary stuff
import subprocess
remove_videos_cmd = f"rm {raw_yt_video_path} {processed_yt_video_path}"
subprocess.run(remove_videos_cmd, shell=True)

Whisper custom tool

In [ ]:
import time
yt_audio.download(output_path=output_dir, filename=yt_audio_filename)

In [ ]:
tools_dir = "../../src/tools"
os.sys.path.append(tools_dir)

In [ ]:
import transcriber

In [ ]:
transcript = transcriber.transcriber.invoke(input={"audio_path": yt_audio_path})

In [ ]:
transcript

In [ ]:
import subprocess
subprocess.run(cmd=f"rm {yt_audio_path}", shell=True)

#### 3. Test Tool

In [ ]:
import os
os.sys.path.append("../../src/")
os.sys.path.append("../../src/agents/")
os.sys.path.append("../../src/utils/")

import react

In [26]:
yt_tasks

,task_id,Question,Level,Final answer,file_name,file_path,Annotator Metadata
4,a1e91b78-d3d8-4675-bb8d-62741b4b68a6,In the video https://www.youtube.com/watch?v=L...,1,3,,,{'Steps': '1. Navigate to the YouTube link. 2....
26,9d191bce-651d-4746-be2d-7ef8ecadb9c2,Examine the video at https://www.youtube.com/w...,1,Extremely,,,{'Steps': '1. Follow the link 2. Watch the cli...
33,0383a3ee-47a7-41a4-b493-519bdefe0488,On the BBC Earth YouTube video of the Top 5 Si...,1,Rockhopper penguin,,,"{'Steps': '1. Search ""top 5 silliest animal mo..."


In [28]:
sample_yt_task = yt_tasks.iloc[0]
sample_yt_question = sample_yt_task["Question"]

In [29]:
sample_yt_question

'In the video https://www.youtube.com/watch?v=L1vXCYZAYYM, what is the highest number of bird species to be on camera simultaneously?'

In [ ]:
react.run_app(user_query=sample_yt_question + ". Just use the pull_youtube_video and the transcriber tools. Try to figure out the number of bird species from the transcript as those are mentioned")

### Web search Tool (Web scrapping)

#### 1. FIlter tasks to solve with the tool

#### 2. Tool PoC

#### 3. Test Tool

### Handle Images Tool (Object detection)

#### 1. FIlter tasks to solve with the tool

#### 2. Tool PoC

#### 3. Test Tool